In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [4]:
df = pd.read_csv("../data/clean/df_cleaned_diag.csv")
df.drop(['Unnamed: 0'],inplace=True,axis=1)

In [5]:
#including diag_1,diag_2,diag_3 columns to test

In [6]:
df['readmitted'] = df['readmitted'].replace({'<30':'yes','>30':'yes'})

In [7]:
features = df.drop(['readmitted'],axis=1)
target = df[['readmitted']]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=5)

In [9]:
# Split the dataset into numerical and categorical column
X_train_cat = X_train.select_dtypes('object')
X_test_cat  = X_test.select_dtypes('object')

X_train_num = X_train.select_dtypes('number')
X_test_num  = X_test.select_dtypes('number')

In [10]:
#Encoding. 


#Decided to use one hot encoding for weight and age  as 
#there's perhaps an ordinal relationship but not well/equally distributed

In [11]:
categorical_column_values = [ df[col].unique() for col in df.select_dtypes('object').columns ]

In [12]:
encoder = OneHotEncoder(drop='first',handle_unknown='ignore', sparse_output=False)

In [30]:
encoder.fit(X_train_cat)
with open('../transformers/transformer_h.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [14]:
X_train_cat_encoded_np = encoder.transform(X_train_cat)
X_test_cat_encoded_np  = encoder.transform(X_test_cat)

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [4, 5, 6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [15]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

In [16]:
X_train_encoded = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test_encoded = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [31]:
normalizer = MinMaxScaler()
normalizer.fit(X_train_encoded)

with open('../scalers/scaler_h.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [18]:
X_train_norm_np = normalizer.transform(X_train_encoded)
X_test_norm_np = normalizer.transform(X_test_encoded)

In [19]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns=X_train_encoded.columns, index=X_train_encoded.index)
X_test_norm_df  = pd.DataFrame(X_test_norm_np,  columns=X_test_encoded.columns,  index=X_test_encoded.index)

In [20]:
###KNN

In [21]:
knn = KNeighborsClassifier(n_neighbors=15)  
knn.fit(X_train_norm_df, y_train)

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=15)

In [32]:
y_pred = knn.predict(X_test_norm_df)

with open('../models/h_KNN.pkl', 'wb') as f:
    pickle.dump(knn, f)


In [23]:
accuracy = knn.score(X_test_norm_df, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5645933014354066


In [24]:
##DEcision Tree 

In [33]:
dt_classifier = DecisionTreeClassifier(random_state=10)
dt_classifier.fit(X_train_norm_df, y_train)

with open('../models/h_DT.pkl', 'wb') as d:
    pickle.dump(dt_classifier, d)

In [26]:
dt_predictions = dt_classifier.predict(X_test_norm_df)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)

Decision Tree Accuracy: 0.5119617224880383


In [27]:
##RF Classifier 

In [34]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=10)
rf_classifier.fit(X_train_norm_df, y_train)

with open('../models/h_RF.pkl', 'wb') as r:
    pickle.dump(rf_classifier, r)

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [35]:
rf_predictions = rf_classifier.predict(X_test_norm_df)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)

Random Forest Accuracy: 0.6028708133971292
